Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function

In [126]:
from graph import Node

def adjust_session(session, frequencies = []):
    if len(session) == 2:
        if session[0] == session[1]:
            session = [session[0]]
            frequencies = [2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 3:
        if session[0] == session[1] and session[0] == session[2]:
            session = session[:1]
            frequencies = [3]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 4:
        if session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1]]
            frequencies = [2, 2]
        else:
            frequencies = [1] * len(session)
    elif len(session) == 5:
        if session[0] == session[1] and session[0] == session[2]:
            session = [session[0], session[3], session[4]]
            frequencies = [3, 1, 1]
        elif session[0] == session[2] and session[1] == session[3]:
            session = [session[0], session[1], session[4]]
            frequencies = [2, 2, 1]
        else:
            frequencies = [1] * len(session)
    else:
        frequencies = [1] * len(session)

    return session, frequencies


def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        # change this to frequency array of length program
        # and add frequency based on index to it    
        freq_addition = []
        _, freq_addition = adjust_session(session=session)

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=freq_addition[0])
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += freq_addition[0]

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = freq_addition[0]
        else:
            cur_node.commands[first_cmd] += freq_addition[0]

        for cmd in range(1, len(session)):

            if cmd >= len(freq_addition):
                    print(session, len(session))
                    print(freq_addition, len(freq_addition))

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=freq_addition[cmd])
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += freq_addition[cmd]
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = freq_addition[cmd]
            else:
                child_node.commands[session[cmd]] += freq_addition[cmd]
            
            cur_node = child_node
        
    return command_dict

Get Prediction Function

In [76]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    return node.get_prediction(previous_command=last_command, num_to_return=result_size), frequencies

Accuracy Function

In [96]:
from thefuzz import fuzz
from thefuzz import process

from statistics import mean

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], fail_frequencies= [], succeed_list = [], succeed_frequencies = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=-1):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    if (graph_depth != -1):
        test_data = [x for x in test_data if len(x) > graph_depth]
    else:
        test_data = [x for x in test_data if len(x) >= 2]
        
    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in tqdm(test_data, leave=False):
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct
        prev5_correct =  correct_with_5

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    succeed_frequencies.append(mean(frequencies))

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                    break

            if prev_correct == correct:
                fail_frequencies.append(mean(frequencies))
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100 and prev_correct == correct:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100 and prev5_correct == correct_with_5:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                              
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.2f} |\n Average Fail Freq: {:.2f}'.format(100 * correct/(test_size-none_count), 100 * correct_with_5/(test_size-none_count), 100 * correct_with_15/(test_size-none_count), 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies)/len(train_data), mean(fail_frequencies)/len(train_data))


In [4]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:06<00:00,  8.78it/s]


All Commands Parse and Replace Args

In [5]:
subset_size = 5

science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, subset_size)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, subset_size)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, subset_size)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, subset_size)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:06<00:00,  8.48it/s]


Predicting last command from 1..n-1 commands

In [36]:
i = -1

print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Scientist session:
 Correct Proportion: 61.87% |
 Correct in 5 not 3: 66.69% |
 Correct in 15 not 5: 74.24% |
 Has Prediction and is Correct: 61.87% |
 Incorrect Proportion: 37.31% |
 None Proportion: 2.15% |
 First Prediction: 47.65% |
 Average Success Freq: 0.14 |
 Average Fail Freq: 0.20 



Experienced session:
 Correct Proportion: 67.66% |
 Correct in 5 not 3: 71.67% |
 Correct in 15 not 5: 78.23% |
 Has Prediction and is Correct: 67.66% |
 Incorrect Proportion: 31.69% |
 None Proportion: 2.03% |
 First Prediction: 52.82% |
 Average Success Freq: 0.23 |
 Average Fail Freq: 0.34 



Non programmer session:
 Correct Proportion: 63.28% |
 Correct in 5 not 3: 68.59% |
 Correct in 15 not 5: 75.80% |
 Has Prediction and is Correct: 63.28% |
 Incorrect Proportion: 36.17% |
 None Proportion: 1.48% |
 First Prediction: 48.34% |
 Average Success Freq: 0.74 |
 Average Fail Freq: 1.06 



Novice session:
 Correct Proportion: 77.05% |
 Correct in 5 not 3: 80.96% |
 Correct in 15 not 5: 86.98% |
 Has Prediction and is Correct: 77.05% |
 Incorrect Proportion: 22.74% |
 None Proportion: 0.91% |
 First Prediction: 58.59% |
 Average Success Freq: 0.24 |
 Average Fail Freq: 0.33 



Accuracy predicting N+1 command from N commands with all commands in graph

In [37]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



Scientist session:
 Correct Proportion: 58.51% |
 Correct in 5 not 3: 65.35% |
 Correct in 15 not 5: 78.97% |
 Has Prediction and is Correct: 58.51% |
 Incorrect Proportion: 41.48% |
 None Proportion: 0.02% |
 First Prediction: 41.58% |
 Average Success Freq: 0.17 |
 Average Fail Freq: 0.24 



Experienced session:
 Correct Proportion: 64.84% |
 Correct in 5 not 3: 71.30% |
 Correct in 15 not 5: 83.42% |
 Has Prediction and is Correct: 64.84% |
 Incorrect Proportion: 35.15% |
 None Proportion: 0.01% |
 First Prediction: 45.70% |
 Average Success Freq: 0.30 |
 Average Fail Freq: 0.41 



Non programmer session:
 Correct Proportion: 60.05% |
 Correct in 5 not 3: 67.01% |
 Correct in 15 not 5: 79.64% |
 Has Prediction and is Correct: 60.05% |
 Incorrect Proportion: 39.94% |
 None Proportion: 0.01% |
 First Prediction: 42.81% |
 Average Success Freq: 0.95 |
 Average Fail Freq: 1.32 



Novice session:
 Correct Proportion: 73.67% |
 Correct in 5 not 3: 79.07% |
 Correct in 15 not 5: 89.26% |
 Has Prediction and is Correct: 73.67% |
 Incorrect Proportion: 26.32% |
 None Proportion: 0.03% |
 First Prediction: 50.31% |
 Average Success Freq: 0.32 |
 Average Fail Freq: 0.43 

Accuracy predicting command 3 from commands 1 through 2:



...

Novice session:
 Correct Proportion: 85.99% |
 Correct in 5 not 3: 89.37% |
 Correct in 15 not 5: 93.93% |
 Has Prediction and is Correct: 85.99% |
 Incorrect Proportion: 13.96% |
 None Proportion: 0.38% |
 First Prediction: 68.33% |
 Average Success Freq: 0.34 |
 Average Fail Freq: 0.45 

Accuracy predicting command 5 from commands 1 through 4:



Scientist session:
 Correct Proportion: 37.39% |
 Correct in 5 not 3: 38.70% |
 Correct in 15 not 5: 39.66% |
 Has Prediction and is Correct: 37.39% |
 Incorrect Proportion: 57.59% |
 None Proportion: 8.02% |
 First Prediction: 30.21% |
 Average Success Freq: 0.21 |
 Average Fail Freq: 0.27 



Experienced session:
 Correct Proportion: 43.29% |
 Correct in 5 not 3: 44.19% |
 Correct in 15 not 5: 44.92% |
 Has Prediction and is Correct: 43.29% |
 Incorrect Proportion: 52.45% |
 None Proportion: 7.52% |
 First Prediction: 35.60% |
 Average Success Freq: 0.35 |
 Average Fail Freq: 0.45 



Non programmer session:
 Correct Proportion: 44.39% |
 Correct in 5 not 3: 46.24% |
 Correct in 15 not 5: 47.64% |
 Has Prediction and is Correct: 44.39% |
 Incorrect Proportion: 52.37% |
 None Proportion: 5.82% |
 First Prediction: 35.47% |
 Average Success Freq: 1.09 |
 Average Fail Freq: 1.41 



Novice session:
 Correct Proportion: 68.38% |
 Correct in 5 not 3: 70.12% |
 Correct in 15 not 5: 71.78% |
 Has Prediction and is Correct: 68.38% |
 Incorrect Proportion: 30.61% |
 None Proportion: 3.20% |
 First Prediction: 54.41% |
 Average Success Freq: 0.34 |
 Average Fail Freq: 0.44 



Consolidated predicting next command. Graph depth of `-1` should accomplish this.

In [17]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    fail_frequencies = []
    succeed_frequencies = []

    print(get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=-1))

    os.makedirs(os.path.dirname("fails/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("fails/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fail_frequencies)

    os.makedirs(os.path.dirname("successes/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("successes/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(succeed_frequencies)


Correct Proportion: 59.32% |
 Correct in 5 not 3: 63.87% |
 Correct in 15 not 5: 71.45% |
 Has Prediction and is Correct: 59.32% |
 Incorrect Proportion: 39.57% |
 None Proportion: 2.71% |
 First Prediction: 44.68% |
 Average Success Freq: 69804 |
 Average Fail Freq: 77479


Correct Proportion: 63.25% |
 Correct in 5 not 3: 67.22% |
 Correct in 15 not 5: 73.90% |
 Has Prediction and is Correct: 63.25% |
 Incorrect Proportion: 35.75% |
 None Proportion: 2.74% |
 First Prediction: 48.08% |
 Average Success Freq: 69131 |
 Average Fail Freq: 72145


Correct Proportion: 64.40% |
 Correct in 5 not 3: 68.95% |
 Correct in 15 not 5: 75.78% |
 Has Prediction and is Correct: 64.40% |
 Incorrect Proportion: 35.02% |
 None Proportion: 1.64% |
 First Prediction: 48.39% |
 Average Success Freq: 81318 |
 Average Fail Freq: 86983


Correct Proportion: 75.19% |
 Correct in 5 not 3: 78.62% |
 Correct in 15 not 5: 84.68% |
 Has Prediction and is Correct: 75.19% |
 Incorrect Proportion: 24.47% |
 None Proportion: 1.37% |
 First Prediction: 56.51% |
 Average Success Freq: 86806 |
 Average Fail Freq: 81448


Save files for all data

In [6]:
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_list = []
        succeed_list = []
        correct_5_not_3 = []
        correct_15_not_5 = []

        get_accuracy(datasets[i], fail_list=fail_list, succeed_list=succeed_list, correct_5_not_3=correct_5_not_3, correct_15_not_5=correct_15_not_5, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(fail_list))

        os.makedirs(os.path.dirname("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(succeed_list))

        os.makedirs(os.path.dirname("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_5_not_3))

        os.makedirs(os.path.dirname("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_15_not_5))


Save frequencies for successes and failures to CSV

In [14]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_frequencies = []
        succeed_frequencies = []

        get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fail_frequencies)

        os.makedirs(os.path.dirname("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(succeed_frequencies)


Sampled Commands Parse and Replace Args

In [25]:
from random import sample
import math

subset_size = 5

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, subset_size)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(0.2 * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, subset_size)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(0.2 * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, subset_size)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(0.2 * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, subset_size)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:02<00:00,  3.73it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [34]:
for i in range(1, 5):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

Accuracy Predicting Command 2 from commands 1 through 1:



Scientist session sample:
 Correct Proportion: 62.75% |
 Correct in 5 not 3: 69.17% |
 Correct in 15 not 5: 82.97% |
 Has Prediction and is Correct: 62.75% |
 Incorrect Proportion: 37.25% |
 None Proportion: 0.00% |
 First Prediction: 45.99% |
 Average Success Freq: 0.41 |
 Average Fail Freq: 0.74


Experienced session sample:
 Correct Proportion: 65.86% |
 Correct in 5 not 3: 73.60% |
 Correct in 15 not 5: 86.97% |
 Has Prediction and is Correct: 65.86% |
 Incorrect Proportion: 34.14% |
 None Proportion: 0.00% |
 First Prediction: 45.30% |
 Average Success Freq: 0.63 |
 Average Fail Freq: 1.09


Non programmer session sample:
 Correct Proportion: 59.77% |
 Correct in 5 not 3: 68.18% |
 Correct in 15 not 5: 83.48% |
 Has Prediction and is Correct: 59.77% |
 Incorrect Proportion: 40.23% |
 None Proportion: 0.00% |
 First Prediction: 41.34% |
 Average Success Freq: 3.59 |
 Average Fail Freq: 6.32


Novice session sample:
 Correct Proportion: 70.98% |
 Correct in 5 not 3: 77.40% |
 Correct in 15 not 5: 87.91% |
 Has Prediction and is Correct: 70.98% |
 Incorrect Proportion: 29.01% |
 None Proportion: 0.01% |
 First Prediction: 46.98% |
 Average Success Freq: 0.51 |
 Average Fail Freq: 0.86
Accuracy Predicting Command 3 from commands 1 through 2:



...

Novice session sample:
 Correct Proportion: 90.72% |
 Correct in 5 not 3: 93.81% |
 Correct in 15 not 5: 97.99% |
 Has Prediction and is Correct: 90.72% |
 Incorrect Proportion: 9.28% |
 None Proportion: 0.04% |
 First Prediction: 72.93% |
 Average Success Freq: 0.49 |
 Average Fail Freq: 0.85
Accuracy Predicting Command 5 from commands 1 through 4:



Scientist session sample:
 Correct Proportion: 79.13% |
 Correct in 5 not 3: 80.96% |
 Correct in 15 not 5: 83.09% |
 Has Prediction and is Correct: 79.13% |
 Incorrect Proportion: 20.77% |
 None Proportion: 0.50% |
 First Prediction: 69.57% |
 Average Success Freq: 0.44 |
 Average Fail Freq: 0.76


Experienced session sample:
 Correct Proportion: 81.25% |
 Correct in 5 not 3: 82.65% |
 Correct in 15 not 5: 83.65% |
 Has Prediction and is Correct: 81.25% |
 Incorrect Proportion: 18.72% |
 None Proportion: 0.16% |
 First Prediction: 72.56% |
 Average Success Freq: 0.64 |
 Average Fail Freq: 1.09


Non programmer session sample:
 Correct Proportion: 78.09% |
 Correct in 5 not 3: 81.61% |
 Correct in 15 not 5: 85.89% |
 Has Prediction and is Correct: 78.09% |
 Incorrect Proportion: 21.80% |
 None Proportion: 0.50% |
 First Prediction: 63.16% |
 Average Success Freq: 3.63 |
 Average Fail Freq: 6.20


Novice session sample:
 Correct Proportion: 84.80% |
 Correct in 5 not 3: 87.42% |
 Correct in 15 not 5: 90.10% |
 Has Prediction and is Correct: 84.80% |
 Incorrect Proportion: 15.17% |
 None Proportion: 0.18% |
 First Prediction: 69.14% |
 Average Success Freq: 0.49 |
 Average Fail Freq: 0.83


Predict N from 1..n-1 in samples

In [33]:
i=-1

print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

Scientist session sample:
 Correct Proportion: 77.59% |
 Correct in 5 not 3: 82.15% |
 Correct in 15 not 5: 89.06% |
 Has Prediction and is Correct: 77.59% |
 Incorrect Proportion: 22.38% |
 None Proportion: 0.12% |
 First Prediction: 63.68% |
 Average Success Freq: 0.35 |
 Average Fail Freq: 0.68


Experienced session sample:
 Correct Proportion: 80.30% |
 Correct in 5 not 3: 84.65% |
 Correct in 15 not 5: 91.14% |
 Has Prediction and is Correct: 80.30% |
 Incorrect Proportion: 19.69% |
 None Proportion: 0.05% |
 First Prediction: 65.05% |
 Average Success Freq: 0.52 |
 Average Fail Freq: 0.95


Non programmer session sample:
 Correct Proportion: 71.33% |
 Correct in 5 not 3: 77.58% |
 Correct in 15 not 5: 88.24% |
 Has Prediction and is Correct: 71.33% |
 Incorrect Proportion: 28.64% |
 None Proportion: 0.11% |
 First Prediction: 54.21% |
 Average Success Freq: 2.95 |
 Average Fail Freq: 5.43


Novice session sample:
 Correct Proportion: 81.20% |
 Correct in 5 not 3: 85.48% |
 Correct in 15 not 5: 91.90% |
 Has Prediction and is Correct: 81.20% |
 Incorrect Proportion: 18.79% |
 None Proportion: 0.06% |
 First Prediction: 62.20% |
 Average Success Freq: 0.40 |
 Average Fail Freq: 0.69


Expand pipes and replace args

In [39]:
science = parser.filter_commands_with_pipe(parser.scientists_commands)
experienced = parser.filter_commands_with_pipe(parser.experienced_commands)
non = parser.filter_commands_with_pipe(parser.non_programmers_commands)
novice = parser.filter_commands_with_pipe(parser.novice_commands)

science_pipes = parser.expand_piped_commands(science)
experienced_pipes = parser.expand_piped_commands(experienced)
non_pipes = parser.expand_piped_commands(non)
novice_pipes = parser.expand_piped_commands(novice)

science_pipes_expand = parser.replace_arg_expanded_pipe(science_pipes)
experienced_pipes_expand = parser.replace_arg_expanded_pipe(experienced_pipes)
non_pipes_expand = parser.replace_arg_expanded_pipe(non_pipes)
novice_pipes_expand = parser.replace_arg_expanded_pipe(novice_pipes)

In [127]:
print("Science pipes metrics:\n", get_accuracy(science_pipes_expand), "\n")
print("Experienced pipes metrics:\n", get_accuracy(experienced_pipes_expand), "\n")
print("Non programmers pipes metrics:\n", get_accuracy(non_pipes_expand), "\n")
print("Novice pipes metrics:\n", get_accuracy(novice_pipes_expand), "\n")

[]


Science pipes metrics:
 Correct Proportion: 84.50% |
 Correct in 5 not 3: 87.34% |
 Correct in 15 not 5: 92.36% |
 Has Prediction and is Correct: 84.50% |
 Incorrect Proportion: 14.95% |
 None Proportion: 3.58% |
 First Prediction: 66.95% |
 Average Success Freq: 0.05 |
 Average Fail Freq: 0.06 



Experienced pipes metrics:
 Correct Proportion: 86.98% |
 Correct in 5 not 3: 93.02% |
 Correct in 15 not 5: 95.81% |
 Has Prediction and is Correct: 86.98% |
 Incorrect Proportion: 12.79% |
 None Proportion: 1.83% |
 First Prediction: 74.43% |
 Average Success Freq: 0.15 |
 Average Fail Freq: 0.14 



Non programmers pipes metrics:
 Correct Proportion: 92.01% |
 Correct in 5 not 3: 93.37% |
 Correct in 15 not 5: 96.49% |
 Has Prediction and is Correct: 92.01% |
 Incorrect Proportion: 7.99% |
 None Proportion: 0.00% |
 First Prediction: 77.19% |
 Average Success Freq: 0.24 |
 Average Fail Freq: 0.15 



Novice pipes metrics:
 Correct Proportion: 90.48% |
 Correct in 5 not 3: 92.86% |
 Correct in 15 not 5: 95.24% |
 Has Prediction and is Correct: 90.48% |
 Incorrect Proportion: 9.09% |
 None Proportion: 4.55% |
 First Prediction: 52.27% |
 Average Success Freq: 2.82 |
 Average Fail Freq: 1.77 

